In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Configurar para reproducibilidad
tf.random.set_seed(42)
np.random.seed(42)

print("TensorFlow version:", tf.__version__)

# 1. CARGAR Y PREPARAR LOS DATOS
print("Cargando dataset CIFAR-10...")
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Nombres de las clases
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
               'dog', 'frog', 'horse', 'ship', 'truck']

print(f"Datos de entrenamiento: {x_train.shape}")
print(f"Datos de prueba: {x_test.shape}")
print(f"Número de clases: {len(class_names)}")

# 2. PREPROCESAR LOS DATOS
# Normalizar píxeles a rango [0,1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convertir etiquetas a categorical
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# 3. VISUALIZAR ALGUNAS IMÁGENES
plt.figure(figsize=(12, 8))
for i in range(16):
    plt.subplot(4, 4, i + 1)
    plt.imshow(x_train[i])
    plt.title(f'{class_names[np.argmax(y_train[i])]}')
    plt.axis('off')
plt.suptitle('Ejemplos del Dataset CIFAR-10')
plt.tight_layout()
plt.show()

# 4. CREAR EL MODELO CNN
def create_cnn_model():
    model = keras.Sequential([
        # Primera capa convolucional
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        
        # Segunda capa convolucional
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        
        # Tercera capa convolucional
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        
        # Aplanar para las capas densas
        layers.Flatten(),
        
        # Capas densas
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3),
        
        # Capa de salida
        layers.Dense(10, activation='softmax')
    ])
    
    return model

# Crear el modelo
model = create_cnn_model()

# 5. COMPILAR EL MODELO
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Mostrar arquitectura del modelo
model.summary()

# 6. CONFIGURAR CALLBACKS
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        patience=5, 
        restore_best_weights=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.2, 
        patience=3, 
        min_lr=0.0001
    )
]

# 7. ENTRENAR EL MODELO
print("\nEntrenando el modelo...")
history = model.fit(
    x_train, y_train,
    batch_size=32,
    epochs=20,  # Puedes aumentar si tienes tiempo
    validation_data=(x_test, y_test),
    callbacks=callbacks,
    verbose=1
)

# 8. EVALUAR EL MODELO
print("\nEvaluando el modelo...")
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"Precisión en test: {test_accuracy:.4f}")
print(f"Loss en test: {test_loss:.4f}")

# 9. VISUALIZAR CURVAS DE ENTRENAMIENTO
plt.figure(figsize=(12, 4))

# Precisión
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.title('Precisión del Modelo')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.legend()
plt.grid(True)

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title('Loss del Modelo')
plt.xlabel('Época')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# 10. HACER PREDICCIONES Y MATRIZ DE CONFUSIÓN
print("\nGenerando predicciones...")
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Reporte de clasificación
print("\nReporte de Clasificación:")
print(classification_report(y_true, y_pred_classes, target_names=class_names))

# Matriz de confusión
plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_true, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.title('Matriz de Confusión')
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# 11. MOSTRAR EJEMPLOS DE PREDICCIONES
def show_predictions(model, x_test, y_test, class_names, num_examples=12):
    predictions = model.predict(x_test)
    
    plt.figure(figsize=(15, 10))
    for i in range(num_examples):
        plt.subplot(3, 4, i + 1)
        plt.imshow(x_test[i])
        
        predicted_class = np.argmax(predictions[i])
        true_class = np.argmax(y_test[i])
        confidence = np.max(predictions[i])
        
        color = 'green' if predicted_class == true_class else 'red'
        plt.title(f'Real: {class_names[true_class]}\n'
                 f'Pred: {class_names[predicted_class]}\n'
                 f'Conf: {confidence:.2f}', color=color)
        plt.axis('off')
    
    plt.suptitle('Ejemplos de Predicciones (Verde=Correcto, Rojo=Incorrecto)')
    plt.tight_layout()
    plt.show()

show_predictions(model, x_test, y_test, class_names)

# 12. GUARDAR EL MODELO (opcional)
print("\nGuardando el modelo...")
model.save('modelo_cnn_cifar10.h5')
print("Modelo guardado como 'modelo_cnn_cifar10.h5'")

print("\n¡Entrenamiento completado!")
print(f"Precisión final: {test_accuracy:.4f}")